In [8]:
import random
import pandas as pd

# Mở rộng tiền tố
prefixes = [
    "Bạn có thể giúp tôi", "Làm sao để", "Tôi nên làm gì khi", "Bạn biết", "Có cách nào",
    "Tại sao lại", "Lúc nào thì", "Chúng ta có thể", "Tôi đang cần", "Làm thế nào để",
    "Nếu tôi muốn", "Gợi ý giúp tôi", "Tôi đang gặp khó khăn trong việc", "Có nên", "Vì sao tôi nên","Shop giới thiệu"
    "Tôi đang băn khoăn về việc", "Mình cần lời khuyên về", "Tôi cần được hướng dẫn", "Giải thích giúp tôi", "Có cái gì hay thì giới thiệu", "Bạn là ai biết", "Tôi là ai biết",
    "Theo bạn thì"
]

# Mở rộng chủ đề
topics = [
    "học tiếng Anh hiệu quả?", "trồng rau sạch tại nhà?", "đi đến Hà Nội nhanh nhất?", "nấu phở bò ngon?",
    "đọc sách nhanh hơn?", "mua vé máy bay giá rẻ?", "liên hệ với tổng đài?", "tìm việc làm online?",
    "giảm căng thẳng sau giờ làm?", "học tốt môn Toán?", "du lịch Đà Lạt tự túc?", "tăng chiều cao tuổi dậy thì?",
    "chăm sóc da mặt đúng cách?", "học lập trình Python?", "giảm cân an toàn?", "nuôi chó tại nhà?",
    "sắp xếp thời gian hiệu quả?", "chọn ngành đại học phù hợp?", "tiết kiệm tiền lương?", "nấu cơm bằng nồi điện?",
    "tìm nhà trọ sinh viên?", "sống xanh và thân thiện với môi trường?", "xem phim hay trên Netflix?",
    "học online không buồn ngủ?", "bảo vệ mắt khi dùng máy tính?", "tăng năng suất làm việc?", 
    "tự học tiếng Trung?", "du lịch nước ngoài lần đầu?", "mua laptop cho sinh viên?", "tập gym cho người mới bắt đầu?", "phim gì hay giới thiệu đi"
]

# Sinh câu hỏi không trùng
questions = set()
while len(questions) < 1000:
    question = f"{random.choice(prefixes)} {random.choice(topics)}"
    # Gắn số ngẫu nhiên để tránh trùng (sẽ loại sau)
    question += f" #{random.randint(1, 1000000)}"
    questions.add(question)

# Loại bỏ phần #random
cleaned_questions = [q.rsplit("#", 1)[0].strip() for q in questions]

# Ghi vào DataFrame
df = pd.DataFrame(cleaned_questions, columns=["text"])
df["label"] = 0  # Nhãn 0: không phải review phim

# Lưu file CSV
df.to_csv("non_review_questions.csv", index=False, encoding="utf-8-sig")
print("Đã tạo xong 1000 câu hỏi không trùng và lưu vào non_review_questions.csv")


Đã tạo xong 1000 câu hỏi không trùng và lưu vào non_review_questions.csv


In [9]:
import pandas as pd

# Đọc file chứa 1000 câu hỏi thông thường (label 0)
non_review_df = pd.read_csv("non_review_questions.csv")

# Đọc file momo review gốc (label 1)
momo_df = pd.read_csv("momo_reviews_balanced.csv")

# Chọn 1000 câu đánh giá phim từ momo (giả sử cột review là 'Noi_dung_sach_giu_dau')
momo_review_df = momo_df[['Noi_dung_sach_giu_dau']].dropna().sample(n=1000, random_state=42)
momo_review_df = momo_review_df.rename(columns={'Noi_dung_sach_giu_dau': 'text'})
momo_review_df['label'] = 1  # Gán nhãn 1 cho câu review phim


In [10]:
# Kết hợp hai tập lại với nhau
combined_df = pd.concat([momo_review_df, non_review_df], ignore_index=True)

# Xáo trộn thứ tự để huấn luyện tốt hơn
combined_df = combined_df.sample(frac=1, random_state=42).reset_index(drop=True)

# Kiểm tra số lượng
print(combined_df['label'].value_counts())  # Nên ra 1000 (label=0) và 1000 (label=1)


label
0    1000
1    1000
Name: count, dtype: int64


In [11]:
combined_df.to_csv("review_detection_dataset.csv", index=False, encoding="utf-8-sig")
print("Đã lưu file review_detection_dataset.csv gồm cả câu hỏi và review phim.")


Đã lưu file review_detection_dataset.csv gồm cả câu hỏi và review phim.


### Training

In [ ]:
# pip install pandas scikit-learn joblib

In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import joblib

# Bước 1: Đọc dữ liệu từ file CSV
df = pd.read_csv("review_detection_dataset.csv")

# Bước 2: Tách dữ liệu và nhãn
X = df["text"]
y = df["label"]

# Bước 3: Chia tập train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Bước 4: Vector hóa bằng TF-IDF
vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Bước 5: Huấn luyện mô hình
model = LogisticRegression()
model.fit(X_train_tfidf, y_train)

# Bước 6: Lưu mô hình và vectorizer
joblib.dump(vectorizer, "review_vectorizer.pkl")
joblib.dump(model, "review_detector_model.pkl")

# Bước 7: Kiểm tra độ chính xác
accuracy = model.score(X_test_tfidf, y_test)
print(f"Accuracy: {accuracy*100:.2f}%")


Accuracy: 99.50%
